### API demo
this is a general notebook for demonstrating the variations available with the circuit pruner api

#### model

In [1]:
#pick a device
device = 'cuda:0'

#pick a model, any pytorch model should do, but well load in our sparsity regularized alexnet model
from circuit_explorer.utils import load_config
config_file = '../configs/alexnet_sparse_config.py'
config = load_config(config_file)
model = config.model
model = model.to(device)


#alternative
#from torchvision import models
#model = models.vgg11(pretrained = True)


from circuit_explorer.utils import convert_relu_layers
convert_relu_layers(model)  #make relus not inplace

/mnt/data/chris/dropbox/Research-Hamblin/Projects/circuit_explorer/env/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/mnt/data/chris/dropbox/Research-Hamblin/Projects/circuit_explorer/env/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/mnt/data/chris/dropbox/Research-Hamblin/Projects/circuit_explorer/env/lib/python3.7/site-packages/torch/cuda/__init__.py:132: UserWarning: 
    Found GPU3 Tesla K40c which is of

layers that can be used a as target for pruning can be identified with the line below;

In [2]:
from circuit_explorer.utils import get_layers_from_model

all_layers = get_layers_from_model(model)
all_layers.keys()

odict_keys(['', 'features', 'features.0', 'features.1', 'features.2', 'features.3', 'features.4', 'features.5', 'features.6', 'features.7', 'features.8', 'features.9', 'features.10', 'features.11', 'features.12', 'avgpool', 'classifier', 'classifier.0', 'classifier.1', 'classifier.2', 'classifier.3', 'classifier.4', 'classifier.5', 'classifier.6'])

all keys in the above dictionary can be used to specify a target layer for pruning

#### feature selection

In [3]:
# specify target feature with unit and layer

import torch

layer = 'classifier.6'   #key from 'all_layers' dictionary 
unit = 1                # single unit feature target, the 2nd dimension (0 is 1st) in the layers space

#OR UNCOMMENT BELOW
#unit = torch.rand(256) # random direction feature in layers latent space ('features.10' is 256 dimensional)  

#### dataloader
our api uses pytorch dataloaders (torch.utils.data.DataLoader) to specify images to use for pruning.
Any DataLoader should do, but we provide some useful data classes as well.

In [4]:
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from circuit_explorer.data_loading import rank_image_data, single_image_data

#if using snip scoring (weight-wise) batch_size must be 1
#batch_size = 1

#for other scoring batch-size can be larger
batch_size = 50


kwargs = {'num_workers': 1, 'pin_memory': True, 'sampler':None} if 'cuda' in device else {}

dataloader = DataLoader(rank_image_data('../image_data/imagenet_2/',class_folders=True),
                        batch_size=batch_size,
                        shuffle=False,
                        **kwargs)


##OR UNCOMMENT BELOW to score with respect to a single image, we provide a simple dataloader class
# image_file_path = '../image_data/imagenet_2/Egyptian_cat/Egyptian_cat_10034.JPEG'
# dataloader = DataLoader(single_image_data(image_file_path),
#                         batch_size=1,
#                         shuffle=False,
#                         **kwargs)


#### score
get saliency scores to target feature from dataloader

In [5]:
#general actgrad scoring
from circuit_explorer.score import actgrad_score, structure_scores, minmax_norm_scores
scores = actgrad_score(model,dataloader,layer,unit)
scores = structure_scores(scores, model, structure='filters')   #structure from  ['kernels','filters']
scores = minmax_norm_scores(scores) 

# #kernel-wise
# from circuit_explorer.score import actgrad_kernel_score
# scores = actgrad_kernel_score(model,dataloader,layer,unit)

# #filter-wise
# from circuit_explorer.score import actgrad_filter_score
# scores = actgrad_filter_score(model,dataloader,layer,unit)

# #weight-wise
# from circuit_explorer.score import snip_score, structured scores
# scores = snip_score(model,dataloader,layer,unit)
    # #convert weight-wise scores to structured scores
# scores = structure_scores(scores, model, structure='kernels')   #structure from  ['kernels','filters']

scores

OrderedDict([('features.0',
              tensor([0.1978, 0.2926, 0.4995, 0.3185, 0.4526, 0.9562, 0.6282, 0.0485, 0.6232,
                      0.1304, 0.5631, 0.6069, 0.5412, 0.4715, 0.5202, 0.0288, 0.4770, 0.8156,
                      0.3955, 0.0529, 0.2566, 0.7679, 0.2396, 0.4641, 0.3726, 0.1949, 0.1888,
                      0.1622, 0.3785, 0.2813, 0.2833, 0.1786, 0.4868, 0.3146, 0.3973, 0.2045,
                      0.3663, 1.0000, 0.4636, 0.2652, 0.2689, 0.4933, 0.4113, 0.3913, 0.2284,
                      0.1669, 0.0759, 0.8656, 0.2756, 0.4831, 0.8277, 0.1296, 0.7697, 0.2644,
                      0.1156, 0.5357, 0.7196, 0.6365, 0.1269, 0.4928, 0.0000, 0.1693, 0.6796,
                      0.2576])),
             ('features.3',
              tensor([0.4617, 0.4942, 0.5675, 0.6108, 0.0678, 0.2549, 0.1487, 0.1510, 0.5193,
                      0.4936, 0.2084, 1.0000, 0.4757, 0.3723, 0.3122, 0.4063, 0.1632, 0.1673,
                      0.0030, 0.4196, 0.3597, 0.7169, 0.8428, 0.2

#### mask
mask low scoring parameters

In [8]:
from circuit_explorer.mask import mask_from_scores, apply_mask, setup_net_for_mask

sparsity = .1
mask = mask_from_scores(scores,sparsity = sparsity, model = model,unit=unit,target_layer=layer)
apply_mask(model,mask) #model now has a weight mask inserted

#reset mask in model to all ones
#setup_net_for_mask(model)

In [9]:
model.features[6].weight_mask.shape

torch.Size([384, 192, 3, 3])